In [1]:
import pandas as pd
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
# evaluation_file = "../verified_data_sample_01_20_2024.xlsx"
evaluation_file = "../verified_data_01_15_2024.xlsx"
df = pd.read_excel(evaluation_file)
df['ground_truths']=df['ground_truths'].apply(lambda x: [x])
df.head(2)

,name,category,title,question,ground_truths,reference,Unnamed: 6
0,Joe Biden,Social Issues,Abortion,Do you support women's right to abortion?,[Yes],NaN,NaN
1,Joe Biden,Social Issues,Gender transition,Should people under the age of 18 years old be...,[Yes],NaN,NaN


In [3]:
from genie_master import GenieMaster

genie_db_path = "./chroma_qadata_db/"
gm = GenieMaster(db_path=genie_db_path)
print("Genie is ready...", gm.model_is_ready())
print("Document collection count...", gm._document_count())

genie=gm.get_genie("\033[4mname\033[0m")
prompt_template = genie.get_prompt_template()
prompt = prompt_template.format_messages(context="\033[4mcontext\033[0m", question="\033[4mquestion?\033[0m")
print("\nGenie's standard prompt template:")
print(f'\033[1mSystem Message:\033[0m {prompt[0].content}')
print(f'\033[1mHuman Message:\033[0m {prompt[1].content}')

Genie Master initialized at: 2024-01-21 00:27:45.282213-05:00
Genie is ready... True
Document collection count... 45990

Genie's standard prompt template:
System Message: You determine how a person will respond to a given question based on that person's quotes
Human Message: Use the following pieces of context to answer the question at the end, and support it with evidence. If you don't know the answer, just say that you don't know, don't try to make up an answer.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"answer": {"title": "Answer", "description": "The response to th

In [4]:
sample_number = 5
df_data = df[['name', 'question', 'ground_truths']].sample(sample_number)
df_data.head(3)

,name,question,ground_truths
27,Joe Biden,Should illegal immigrants have access to gover...,[Yes]
40,Joe Biden,Should the U.S. expand offshore oil drilling?,[No]
35,Joe Biden,Should the electoral college be abolished?,[No]


In [5]:
df_eval = gm.evaluate(df_data)

eval_result = dict()
df_scores = df_eval.drop(
    columns=["name", "question", "answer", "contexts", "ground_truths"]
)

for metric in df_scores.columns:
    eval_result[metric] = df_scores[metric].sum() / df_scores.shape[0]

print('')
for metric, score in eval_result.items():
    print(f"{metric}: {score}")

Evaluating <Genie name=Joe Biden model=gpt-3.5-turbo-1106> 
Getting LLM response...
evaluating with [answer_similarity]


100%|██████████| 1/1 [00:00<00:00,  1.66it/s]



answer_similarity: 0.7477570763090058


In [6]:
df_eval

,name,question,answer,contexts,ground_truths,answer_similarity
0,,Should illegal immigrants have access to gover...,"{""answer"": ""unknown"", ""reasoning"": ""There is n...",[people who would otherwise qualify for Medica...,[Yes],0.731151
1,,Should the U.S. expand offshore oil drilling?,"{""answer"": ""no"", ""reasoning"": ""Joe Biden has e...",[End offshore drillingwww.politico.com/2020-el...,[No],0.754785
2,,Should the electoral college be abolished?,"{""answer"": ""unknown"", ""reasoning"": ""There is n...",[supports giving voting representation to the ...,[No],0.756009
3,,Should the government increase funding for men...,"{""answer"": ""yes"", ""reasoning"": ""Joe Biden has ...",[Get people who should be supported with socia...,[Yes],0.745678
4,,Should the government give tax credits and sub...,"{""answer"": ""yes"", ""reasoning"": ""Joe Biden has ...",[emissions.•\tIncluded in the 2009 American Re...,[Yes],0.751162


In [7]:
from datetime import datetime
# store result
eval_records_folder_path = "../evaluation_records/"

now = datetime.now()
path = eval_records_folder_path + now.strftime("%m%d%Y%H%M") + ".xlsx"
df_eval.to_excel(path, index=False)
path

'../evaluation_records/012120240027.xlsx'